In [17]:
import pandas as pd
import os
from dotenv import load_dotenv  # If using python-dotenv
from streetview import get_streetview
import asyncio
import aiofiles
from functools import partial

In [18]:
load_dotenv()
api_key = os.environ.get("API")
# print(api_key)

folder_save = 'raw'

#was started at 12k

api_calls_limit = 22000 #SHOULD BE MAX 10000
print(f"api calls limit: {api_calls_limit}")

api calls limit: 22000


In [19]:
# empty_df = pd.DataFrame(columns=['downloaded'])

# # Save the DataFrame to a CSV file
# empty_df.to_csv('streetviewimages.csv', index=False)

# print("Created empty streetviewimages.csv with 'downloaded' column")

In [20]:
panoid_df = pd.read_csv('pano_id_filtered.csv')
panoid_df

,panoid
0,DKl7yRDcP00oKid6jD3MZg
1,PX9xBtoClYNfHJRyenY3eQ
2,ByQt67gS0FKLmbBFgc0LrA
3,I7MoLOO_oZ92FCodbI2ThA
4,o639-1IzUKFRvOHkoHWQew
...,...
145241,frIc7lEl-YVCuq4utP678A
145242,iabyem1ABuYMqMnpa2rI7Q
145243,cnX4N8LYBMsYMA-Zjog_bw
145244,_abi6yQJcZwST4dBpcj1rQ


In [21]:
# Get downloaded images from the folder instead of CSV
downloaded_set = set()
if os.path.exists(folder_save):
    for filename in os.listdir(folder_save):
        if filename.endswith('.jpeg'):
            # Extract the panoid from the filename (remove .jpeg extension)
            panoid = filename.replace('.jpeg', '')
            downloaded_set.add(panoid)

len(downloaded_set)

122414

In [22]:
# panoid_ls = panoid_df['panoid'].tolist()


# panoidnotyet_ls = [panoid for panoid in panoid_ls if panoid not in downloaded_set]
# print(len(panoidnotyet_ls))

# for i in range(api_calls_limit):
#   pano_id = panoidnotyet_ls[i]
#   image = get_streetview(pano_id=pano_id, api_key=api_key)
#   image.save(f"{folder_save}/{pano_id}.jpeg", "jpeg")


In [23]:
# panoid_ls = panoid_df['panoid'].tolist()


# panoidnotyet_ls = [panoid for panoid in panoid_ls if panoid not in downloaded_set]
# print(len(panoidnotyet_ls))


# async def download_streetview(pano_id, api_key, folder_save):
#   image = get_streetview(pano_id=pano_id, api_key=api_key)
#   image_path = f"{folder_save}/{pano_id}.jpeg"
#   image.save(image_path, "jpeg")
  
#   new_panos_df = pd.DataFrame({
#       'downloaded': [pano_id],
#   })
#   async with aiofiles.open('streetviewimages.csv', 'a') as f:
#       await f.write(f"{pano_id}\n")
  
#   return pano_id

# async def main():
#   tasks = []
#   for i in range(api_calls_limit):
#     pano_id = panoidnotyet_ls[i]
#     task = asyncio.create_task(download_streetview(pano_id, api_key, folder_save))
#     tasks.append(task)
  
#   completed = await asyncio.gather(*tasks)
#   print(f"Downloaded {len(completed)} images")

# # Run the async code
# await main()


In [ ]:
panoid_ls = panoid_df['panoid'].tolist()


panoidnotyet_ls = [panoid for panoid in panoid_ls if panoid not in downloaded_set]
print(len(panoidnotyet_ls))

async def download_streetview(pano_id, api_key, folder_save):
  try:
    image = get_streetview(pano_id=pano_id, api_key=api_key)
    # Convert RGBA to RGB if needed
    if image.mode == 'RGBA':
      image = image.convert('RGB')
    image.save(f"{folder_save}/{pano_id}.jpeg", "jpeg")
    return pano_id
  except Exception as e:
    print(f"Error downloading {pano_id}: {str(e)}")
    return None

async def main():
  batch_size = 100
  total_downloads = min(api_calls_limit, len(panoidnotyet_ls))
  
  for start_idx in range(0, total_downloads, batch_size):
    end_idx = min(start_idx + batch_size, total_downloads)
    batch_ids = panoidnotyet_ls[start_idx:end_idx]
    
    tasks = []
    for pano_id in batch_ids:
      task = asyncio.create_task(download_streetview(pano_id, api_key, folder_save))
      tasks.append(task)
    
    completed = await asyncio.gather(*tasks)
    # Filter out None values (failed downloads)
    completed = [p for p in completed if p is not None]
    print(f"Downloaded batch {start_idx//batch_size + 1}: {len(completed)} images")

# Run the async code
await main()


22832
